In [1]:
%pylab inline
import os
import copy
import json
import cv2
#import matplotlib
import torch
import torch.nn as nn
from pytorch_grad_cam import GradCAM
#from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50, inception_v3, vgg16
from models import RadFormer 
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from dataloader import GbUsgDataSet
import seaborn as sns

from torchvision.io import ImageReadMode

from torch.nn.functional import softmax, interpolate
from torchvision.io.image import read_image
from torchvision.models import resnet18
from torchvision.transforms.functional import normalize, resize, to_pil_image

from torchcam.methods import SmoothGradCAMpp
from torchcam.utils import overlay_mask

Populating the interactive namespace from numpy and matplotlib


/home/soumen/anaconda3/envs/url/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/soumen/anaconda3/envs/url/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/soumen/anaconda3/envs/url/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/soumen/anaconda3/envs/url/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
model = RadFormer(local_net="bagnet33", \
                        global_weight=0.55, \
                        local_weight=0.1, \
                        fusion_weight=0.35, \
                        use_rgb=True, num_layers=4, pretrain=False).cuda()
#model_file="./model_weights/design5/trans_attn_bag_test.pkl"
model_file="model_weights/radformer/radformer.pkl"
model.load_state_dict(torch.load(model_file))
model.float().cuda()

RadFormer(
  (global_branch): Resnet50(
    (net): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inp

In [3]:
model.eval()

cam_extractor = SmoothGradCAMpp(model, target_layer=model.global_branch.net.layer4[-1])

In [3]:
normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])

val_dataset = GbUsgDataSet(data_dir="../XGC-data/imgs/", 
                        image_list_file="../XGC-data/xgc-cls.txt",
                        transform=transforms.Compose([
                            transforms.Resize(224),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            #normalize,
                        ]))

val_loader = DataLoader(dataset=val_dataset, batch_size=1, 
                                shuffle=False, num_workers=0)

In [38]:
def plot(model, cam_extractor, loader, save_dir, pred_file):
    
    LABEL = ["normal", "benign (xgc)", "malignant"]
    os.makedirs(save_dir, exist_ok=True)
    preds = {}
    with open(pred_file, "r") as f:
        for line in f.readlines():
            name, gt, pred, score = line.strip().split(",")
            preds[name] = int(pred)
        
    for im, label, filename in loader:
        fname = "/".join(filename[0].split("/")[-2:])
        print(fname)
        folder = fname.split("/")[-2]
        pred_label_text = LABEL[preds[folder]]

        os.makedirs("%s/%s"%(save_dir, folder), exist_ok=True)
            
        g_out = model(im.cuda())
        
        _, pred = torch.max(g_out, dim=1)
        
        pred_label = pred.item()
        #pred_label_text = LABEL[pred_label]
        gt = LABEL[label.item()]
        
        cam = cam_extractor(pred_label, g_out)[0]

        result = overlay_mask(to_pil_image(im.squeeze(0)), to_pil_image(cam.squeeze(0), mode='F'), alpha=0.5)
       
        orig = to_pil_image(im.squeeze(0))
        vis = get_concat_h(orig, result) 
        I = ImageDraw.Draw(vis)
        myfont = ImageFont.truetype('FreeMono.ttf', 15)
        I.text((20, 180), "true: %s"%gt, font=myfont, fill=(255, 0, 0))
        I.text((im.squeeze(0).size()[2]+20, 180), "pred: %s"%pred_label_text, font=myfont, fill=(0, 255, 0))

        vis.save("%s/%s"%(save_dir, fname))
       
        

In [39]:
plot(model, cam_extractor, val_loader, "vis-radformer-2", "radformer.csv")

21/IMG-0001-00003.jpg


/home/soumen/anaconda3/envs/url/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


21/IMG-0001-00002.jpg
21/IMG-0001-00001.jpg
21/IMG-0001-00004.jpg
21/IMG-0001-00005.jpg
21/IMG-0001-00006.jpg
21/IMG-0001-00007.jpg
21/IMG-0001-00011.jpg
21/IMG-0001-00009.jpg
21/IMG-0001-00008.jpg
21/IMG-0001-00012.jpg
21/IMG-0001-00013.jpg
21/IMG-0001-00010.jpg
32/IMG-0005-00001.jpg
32/IMG-0005-00008.jpg
32/IMG-0005-00015.jpg
32/IMG-0005-00020.jpg
32/IMG-0005-00019.jpg
32/IMG-0005-00003.jpg
32/IMG-0005-00009.jpg
32/IMG-0005-00018.jpg
32/IMG-0005-00002.jpg
32/IMG-0005-00006.jpg
32/IMG-0005-00011.jpg
32/IMG-0005-00014.jpg
32/IMG-0005-00004.jpg
32/IMG-0005-00012.jpg
32/IMG-0005-00017.jpg
32/IMG-0005-00010.jpg
32/IMG-0005-00013.jpg
32/IMG-0005-00005.jpg
32/IMG-0005-00016.jpg
32/IMG-0005-00007.jpg
38/IMG-0001-00001.jpg
38/IMG-0001-00002.jpg
38/IMG-0001-00008.jpg
38/IMG-0001-00004.jpg
38/IMG-0001-00005.jpg
38/IMG-0001-00007.jpg
38/IMG-0001-00003.jpg
38/IMG-0001-00009.jpg
38/IMG-0001-00006.jpg
38/IMG-0001-00012.jpg
38/IMG-0001-00010.jpg
38/IMG-0001-00014.jpg
38/IMG-0001-00013.jpg
38/IMG-000

In [4]:
from PIL import Image, ImageDraw, ImageFont
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [4]:
def plot_cam(model, cam_extractor, read_file, outdir):
    LABEL = ["normal", "xgc", "malignant"]
    with open(read_file, "r") as f:
        for line in f.readlines():
            fname, gt = line.strip().split(",")
            name = "/".join(fname.split("/")[-2:])
            folder = fname.split("/")[-2]
            os.makedirs("%s/%s"%(outdir, folder), exist_ok=True)
            gt = LABEL[int(gt)]
            #pred = LABEL[int(pred)]
            img = read_image("../XGC-data/imgs/%s"%fname, ImageReadMode.RGB)
            # Preprocess it for your chosen model
            input_tensor = resize(img, (224, 224)) / 255.#, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            input_tensor = input_tensor.unsqueeze(0).cuda()
            g_out = model(input_tensor)
            pred = g_out.squeeze(0).argmax().item()
            cam = cam_extractor(pred, g_out)[0]
            #for name, cam in zip(cam_extractor.target_names, cams):
            result = overlay_mask(to_pil_image(img), to_pil_image(cam.squeeze(0), mode='F'), alpha=0.5)
            #orig = to_pil_image(img)
            vis = result #get_concat_h(orig, result) 
            #I = ImageDraw.Draw(vis)
            #myfont = ImageFont.truetype('FreeMono.ttf', 65)
            #I.text((20, 40), "true: %s"%gt, font=myfont, fill=(255, 0, 0))
            #pred_label = LABEL[int(pred)]
            #I.text((img.size()[2]+20, 40), "pred: %s"%pred_label, font=myfont, fill=(0, 255, 0))

            vis.save("%s/%s"%(outdir, name))

        

In [5]:
plot_cam(model, cam_extractor, "../XGC-data/xgc-cls.txt", "vis-radformer")

/home/soumen/anaconda3/envs/url/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
